In [31]:
from typing import *
import libcst as cst
import networkx as nx
import pandas as pd
import numpy as np

In [32]:
from alkh import cst_utils

In [33]:
# file_path = 'play.py'
file_path = 'play2.py'
# file_path = '/mnt/dev/azure_projects/ATool/core/aoi_defects_clustering/aoi_defect_clustering.py'

In [34]:
file_lines = open(file_path, 'r').readlines()
file_content = open(file_path, 'r').read()
wrapper = cst.metadata.MetadataWrapper(cst.parse_module(file_content))
scopes = set(wrapper.resolve(cst.metadata.ScopeProvider).values())
ranges = wrapper.resolve(cst.metadata.PositionProvider)
file_number_of_lines = len(file_lines)

In [35]:
class AssignCollector(cst.CSTVisitor):
    def __init__(self, ranges):
        super().__init__()
        self._ranges = ranges
        self._assign_info: List[Tuple] = []

    def get_info(self):
        return self._assign_info

    def visit_Assign(self, node: cst.FunctionDef) -> None:
        pos_start = self._ranges[node].start
        pos_end = self._ranges[node].end
        value_collector = ValueCollector()
        node.value.visit(value_collector)
        value_dict = {'names': value_collector.names, 'ints': value_collector.ints, 'floats': value_collector.floats}
        for target in node.targets:
            target_collector = ValueCollector()
            target.visit(target_collector)
            names_list = target_collector.names
            if names_list:
                self._assign_info.append((target_collector.names, value_dict, pos_start.line, pos_end.line))


class ValueCollector(cst.CSTVisitor):
    def __init__(self):
        super().__init__()
        self.names: List[List] = []
        self.ints: List[str] = []
        self.floats: List[str] = []
        self._attribute_level = 0

    def visit_Name(self, node: cst.FunctionDef) -> None:
        if self._attribute_level == 0:
            self.names.append([node.value])

    def visit_Integer(self, node: cst.FunctionDef) -> None:
        self.ints.append(node.value)

    def visit_Float(self, node: cst.FunctionDef) -> None:
        self.floats.append(node.value)

    def visit_Attribute(self, node: cst.FunctionDef) -> None:
        self._attribute_level += 1
        
    def visit_Subscript(self, node: cst.FunctionDef) -> None:
        self._attribute_level += 1

    def leave_Attribute(self, node: cst.FunctionDef) -> None:
        if isinstance(node.value, cst.Name):
            self.names.append([node.value.value, node.attr.value])
        else:
            self.names[len(self.names) - 1].append(node.attr.value)
        self._attribute_level -= 1
        
    def leave_Subscript(self, node: cst.FunctionDef) -> None:
        if isinstance(node.value, cst.Name):
            self.names.append([node.value.value])
        self._attribute_level -= 1

In [36]:
assign_collector = AssignCollector(ranges)
wrapper.visit(assign_collector)
assign_collector.get_info()

[]

In [37]:
wrapper.module.body[0].body[0]

TypeError: 'IndentedBlock' object is not subscriptable

In [68]:
file_lines = open(file_path, 'r').readlines()
file_content = open(file_path, 'r').read()
wrapper = cst.metadata.MetadataWrapper(cst.parse_module(file_content))
scopes = set(wrapper.resolve(cst.metadata.ScopeProvider).values())
ranges = wrapper.resolve(cst.metadata.PositionProvider)
file_number_of_lines = len(file_lines)

In [69]:
scopes

{<libcst.metadata.scope_provider.GlobalScope at 0x7f388e1c4cd0>}

In [70]:
k = wrapper.module.body[0]
k

If(
    test=Integer(
        value='1',
        lpar=[],
        rpar=[],
    ),
    body=IndentedBlock(
        body=[
            SimpleStatementLine(
                body=[
                    Pass(
                        semicolon=MaybeSentinel.DEFAULT,
                    ),
                ],
                leading_lines=[],
                trailing_whitespace=TrailingWhitespace(
                    whitespace=SimpleWhitespace(
                        value='',
                    ),
                    comment=None,
                    newline=Newline(
                        value=None,
                    ),
                ),
            ),
        ],
        header=TrailingWhitespace(
            whitespace=SimpleWhitespace(
                value='',
            ),
            comment=None,
            newline=Newline(
                value=None,
            ),
        ),
        indent=None,
        footer=[],
    ),
    orelse=If(
        test=Integer(
            value

In [71]:
type(k)

libcst._nodes.statement.If

In [72]:
ranges[k]

CodeRange(start=CodePosition(line=1, column=0), end=CodePosition(line=11, column=8))

In [73]:
k.test

Integer(
    value='1',
    lpar=[],
    rpar=[],
)

In [74]:
ranges[k.test]

CodeRange(start=CodePosition(line=2, column=8), end=CodePosition(line=2, column=9))

In [75]:
ranges[k.test].end.line

2

In [76]:
cst.If

libcst._nodes.statement.If

In [90]:
class IfElseScopeCollector(cst.CSTVisitor):
    def __init__(self):
        super().__init__()
        self.scopes: List[Tuple] = []

    def visit_If(self, node: cst.If) -> None:
        node_range = ranges[node]
        start_line = node_range.start.line
        head_end_line = ranges[node.whitespace_after_test].end.line
        end_line = node_range.end.line
        self.scopes.append((start_line, head_end_line, end_line))
        
    def visit_Else(self, node: cst.If) -> None:
        node_range = ranges[node]
        start_line = node_range.start.line
        head_end_line = ranges[node.whitespace_before_colon].end.line
        end_line = node_range.end.line
        self.scopes.append((start_line, head_end_line, end_line))

In [91]:
collector = IfElseScopeCollector()
m = wrapper.visit(collector)

In [92]:
collector.scopes

[(1, 2, 11), (4, 4, 11), (6, 7, 11), (9, 10, 11)]